In [1]:
import pandas as pd

In [ ]:
data = pd.read_csv("C:/Users/Quang Anh/Desktop/job_skills.csv")

data.head()

In [2]:
data['keywords'] = data['job_link'].str.extract('view/(.*)')[0].str.split('-at-').str[0].str.split('-')

In [3]:
data.drop('job_link', axis=1, inplace=True)

In [ ]:
data.head()

In [5]:
titles = [
    'data', 'finance', 'financial', 'business',
    'accountant', 'analyst', 'architect', 'auditor', 'barista',
    'chef', 'consultant', 'conductor' 'designer', 'developer', 'economist',
    'editor', 'engineer', 'instructor', 'operator', 'journalist',
    'lawyer', 'manager', 'marketer', 'nurse', 'pharmacist',
    'photographer', 'planner', 'programmer', 'psychologist', 'recruiter',
    'scientist', 'surgeon', 'therapist', 'trainer', 'writer',
    'actuary', 'broker', 'butler', 'cashier', 'custodian',
    'dietitian', 'electrician', 'farmer', 'gardener', 'hygienist',
    'illustrator', 'judge', 'librarian', 'mason', 'navigator',
    'optometrist', 'paramedic', 'quartermaster', 'ranger', 'sculptor',
    'tailor', 'underwriter', 'valet', 'welder', 'xylographer',
    'yeoman', 'zoologist'
]

In [6]:
# Filtering the DataFrame
data['filtered_entries'] = data['keywords'].apply(lambda x: [word for word in x if word in titles])

In [48]:
# Removing rows with no matches (empty lists)
filtered_jobs = data[data['filtered_entries'].map(bool)]

In [ ]:
filtered_jobs.head()

In [52]:
from collections import Counter
import numpy as np

# EDA Step 1: Frequency Distribution of job titles
job_title_counts = Counter([job for entry in filtered_jobs['filtered_entries'] for job in entry])

# EDA Results
job_title_counts

most_common_titles = job_title_counts.most_common()  # This will sort job titles by frequency in descending order
print(most_common_titles)

[('manager', 257130), ('nurse', 113513), ('engineer', 81771), ('analyst', 34740), ('consultant', 20781), ('accountant', 12337), ('scientist', 11449), ('therapist', 11118), ('instructor', 9623), ('architect', 9465), ('developer', 8563), ('chef', 8526), ('planner', 6737), ('trainer', 5981), ('cashier', 5032), ('operator', 3933), ('psychologist', 3743), ('surgeon', 3636), ('auditor', 2963), ('writer', 2635), ('recruiter', 2053), ('pharmacist', 1805), ('electrician', 1621), ('underwriter', 1364), ('editor', 1111), ('programmer', 1004), ('lawyer', 925), ('barista', 816), ('navigator', 675), ('broker', 512), ('custodian', 452), ('welder', 443), ('hygienist', 322), ('photographer', 270), ('dietitian', 248), ('paramedic', 242), ('optometrist', 225), ('actuary', 148), ('gardener', 121), ('librarian', 116), ('economist', 102), ('valet', 99), ('mason', 84), ('butler', 71), ('marketer', 60), ('journalist', 46), ('judge', 44), ('ranger', 35), ('illustrator', 26), ('tailor', 12), ('farmer', 11), ('z

In [2]:
desc_1 = pd.read_csv("C:/Users/Quang Anh/Desktop/CourseraDataset-Clean.csv")


In [3]:
desc_1 = desc_1.loc[(desc_1['What you will learn'].notna()) & 
                               (desc_1['What you will learn'] != 'Not specified')]

In [4]:
desc_1 = desc_1[['Course Title', 'What you will learn']]

In [5]:
desc_2 = pd.read_csv("C:/Users/Quang Anh/Downloads/Telegram Desktop/coursera_courses (2).csv")

In [6]:
desc_2 = desc_2.loc[(desc_2['course_description'].notna())]
desc_2 = desc_2[['course_title', 'course_description']]

In [7]:
desc_3 = pd.read_csv("C:/Users/Quang Anh/Downloads/coursera_course_dataset_v3.csv") 


In [8]:
desc_3 = desc_3[['Title', 'course_description']]
desc_3 = desc_3.loc[(desc_3['course_description'].notna())]

In [9]:
desc_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 402 entries, 0 to 617
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Title               402 non-null    object
 1   course_description  402 non-null    object
dtypes: object(2)
memory usage: 9.4+ KB


In [10]:
courses_data_cleaned = pd.read_csv("C:/Users/Quang Anh/Downloads/Telegram Desktop/final_courses.csv")

In [28]:
courses_data_cleaned.head()

,reviews,reviewers,rating,name,institution,Overall Ratings,Level,Duration,Num of Reviews,Skill gain,Instructor,Demeaned Rating,Fee
0,"The course was very useful, and it covered all...",1,5,Structuring Machine Learning Projects,DeepLearning.AI,4.8,Beginner level,6,49,"Decision-Making, Machine Learning, Deep Learni...","Younes Bensouda Mourri', 'Kian Katanforoosh', ...",0.00,49
1,...,7,5,Introduction to Data Analysis Using Excel,Rice University,4.7,None,24,10,"Lookup Table, Data Analysis, Microsoft Excel, ...",Sharad Borle,0.00,49
2,Very good refresher for Microsoft Excel,1017,5,Excel Skills for Business: Essentials,Macquarie University,4.9,Beginner level,26,47,"Graphs, Spreadsheet, Microsoft Excel, Chart","Nicky Bull', 'Professor Yvonne Breyer', 'Dr Pr...",0.00,49
3,I like the week 3 and week 4. Help me to revie...,2020,5,Data Science Math Skills,Duke University,4.5,Beginner level,13,11,"""Bayes Theorem"", Bayesian Probability, Probabi...","Paul Bendich', 'Daniel Egger",0.25,49
4,awesome!!!!!,2020,5,Using Python to Access Web Data,University of Michigan,4.8,Beginner level,19,43,"Json, Xml, Python Programming, Web Scraping",Charles Russell Severance,0.25,49


In [80]:
merged_df = courses_data_cleaned.merge(desc_1, how='left', left_on='name', right_on='Course Title')
merged_df = merged_df.merge(desc_2, how='left', left_on='name', right_on='course_title')
merged_df = merged_df.merge(desc_3, how='left', left_on='name', right_on='Title')

In [81]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 608654 entries, 0 to 608653
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   reviews               608591 non-null  object 
 1   reviewers             603398 non-null  object 
 2   rating                608654 non-null  int64  
 3   name                  608654 non-null  object 
 4   institution           608654 non-null  object 
 5   Overall Ratings       608654 non-null  float64
 6   Level                 608654 non-null  object 
 7   Duration              608654 non-null  int64  
 8   Num of Reviews        608654 non-null  int64  
 9   Skill gain            608654 non-null  object 
 10  Instructor            608654 non-null  object 
 11  Demeaned Rating       608654 non-null  float64
 12  Fee                   608654 non-null  int64  
 13  Course Title          329839 non-null  object 
 14  What you will learn   329839 non-null  object 
 15  

In [82]:
merged_df.drop(['Course Title', 'course_title', 'Title'], axis=1, inplace=True)

In [83]:
unique_merged_df = merged_df.dropna(subset=['course_description_x', 'course_description_y', 'What you will learn'], how='all')
unique_merged_df = unique_merged_df.drop_duplicates(subset='name')
num_rows = unique_merged_df.shape[0]
print(num_rows)

309


In [15]:
unique_merged_df.head()

,reviews,reviewers,rating,name,institution,Overall Ratings,Level,Duration,Num of Reviews,Skill gain,Instructor,Demeaned Rating,Fee,What you will learn,course_description_x,course_description_y
0,"The course was very useful, and it covered all...",1,5,Structuring Machine Learning Projects,DeepLearning.AI,4.8,Beginner level,6,49,"Decision-Making, Machine Learning, Deep Learni...","Younes Bensouda Mourri', 'Kian Katanforoosh', ...",0.00,49,NaN,In the third course of the Deep Learning Speci...,NaN
2,Very good refresher for Microsoft Excel,1017,5,Excel Skills for Business: Essentials,Macquarie University,4.9,Beginner level,26,47,"Graphs, Spreadsheet, Microsoft Excel, Chart","Nicky Bull', 'Professor Yvonne Breyer', 'Dr Pr...",0.00,49,NaN,Gain Skills to understand the fundamentals of ...,NaN
3,I like the week 3 and week 4. Help me to revie...,2020,5,Data Science Math Skills,Duke University,4.5,Beginner level,13,11,"""Bayes Theorem"", Bayesian Probability, Probabi...","Paul Bendich', 'Daniel Egger",0.25,49,NaN,This Specialization covers the concepts and to...,This Specialization covers the concepts and to...
4,awesome!!!!!,2020,5,Using Python to Access Web Data,University of Michigan,4.8,Beginner level,19,43,"Json, Xml, Python Programming, Web Scraping",Charles Russell Severance,0.25,49,Use regular expressions to extract data from s...,This course will show how one can treat the In...,NaN
6,very good!,2020,5,Introduction to Programming with MATLAB,Vanderbilt University,4.8,Beginner level,35,17,"Computer Programming, Problem Solving, Matlab,...","Mike Fitzpatrick', 'Akos Ledeczi",0.25,49,You will learn fundamental computer programmin...,This course introduces you to sampling and exp...,NaN


In [84]:
def merge_columns(row):

    x = row['course_description_x'] if pd.notna(row['course_description_x']) else ''
    y = row['course_description_y'] if pd.notna(row['course_description_y']) else ''
    z = row['What you will learn'] if pd.notna(row['What you will learn']) else ''
    
    if x == y:
        return x + ' ' + '\n' + ' ' + z
    elif x == z:
        return x + ' ' + '\n' + ' ' + y
    elif y == z:
        return y + ' ' + '\n' + ' ' + x
    else:
        return x + ' ' + '\n' + ' ' + y + ' ' + '\n' + ' ' + z

unique_merged_df['merged_description'] = unique_merged_df.apply(merge_columns, axis=1)
unique_merged_df = unique_merged_df.reset_index(drop=True)
unique_merged_df['course_id'] = unique_merged_df.index

In [85]:
courses = unique_merged_df[['course_id', 'name', 'merged_description']]

In [141]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

additional_stopwords = ['learn', 'course', 'students', 'introduction', 'understand', 'overview', 
    'concepts', 'designed', 'including', 'using', 'used', 'provides', 
    'students', 'aimed', 'teach', 'taught', 'include', 'includes', 'including', 
    'understanding', 'module', 'modules', 'cover', 'covers', 'covered', 
    'introduce', 'introduces', 'introduced', 'overview', 'knowledge', 'skills', 
    'learning', 'lecture', 'lectures', 'topic', 'topics', 'area', 'areas', 
    'focus', 'focuses', 'focused', 'approach', 'approaches', 'principles', 
    'principle', 'content', 'contents', 'key', 'features', 'feature', 
    'element', 'elements', 'basis', 'basic', 'basics', 'foundation', 'foundations',
    'beginner', 'intermediate', 'advanced', 'level', 'levels', 'intended',
    'objective', 'objectives', 'goal', 'goals', 'outcome', 'outcomes', 'aim', 'aims',
    'gain', 'gains', 'apply', 'effective', 'efficient', 'specialize', 'specialization',
    'specialise', 'certificate', 'certification', 'you', 'will', 'need',
    'want', 'have', 'able', 'well', 'one', 'two', 'three', 'first', 'second',
    'third', 'success', 'successful', 'sucessfully', 'like', 'many' , 'much', 'also', 'use', 'uses', 'used',
    'work', 'works', 'working', 'workshop', 'workshops', 'provide', 'provides', 'provided']

all_stop_words = stopwords.words('english') + additional_stopwords



[nltk_data] Downloading package punkt to C:\Users\Quang
[nltk_data]     Anh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Quang
[nltk_data]     Anh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Quang
[nltk_data]     Anh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [142]:
# Initialize the PorterStemmer
stemmer = PorterStemmer()

# Preprocessing function with N-gram consideration
def preprocess_text(text, lemmatizer=WordNetLemmatizer(), stop_words=all_stop_words):
    text = text.lower()  # Lowercasing
    # text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    tokens = word_tokenize(text)  # Tokenization
    alphabetic_tokens = [token for token in tokens if re.match("^[a-zA-Z]+$", token)] # check for alphabetic tokens
    filtered_tokens = [word for word in alphabetic_tokens if word not in stop_words]  # Remove stopwords
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]  # Lemmatization
    return ' '.join(lemmatized_tokens)

In [143]:

# If 'courses' might be a slice of another DataFrame, explicitly create a copy to work on:
courses = courses.copy()

# Now, safely apply the preprocessing and assign to the 'processed_description' column
courses['processed_description'] = courses['merged_description'].apply(preprocess_text)


In [99]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Assuming courses['processed_description'] is your preprocessed text
embeddings = model.encode(courses['processed_description'].tolist())


In [108]:
from sklearn.decomposition import PCA

# Let's assume we want to reduce to 50 dimensions
pca = PCA(n_components=50)
reduced_embeddings = pca.fit_transform(embeddings)

In [109]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

# Calculate cosine similarity
cosine_sim = cosine_similarity(reduced_embeddings)

# Normalize Euclidean distances to be between 0 and 1
max_dist = np.max(euclidean_distances(reduced_embeddings))
normalized_euclidean_sim = 1 - (euclidean_distances(reduced_embeddings) / max_dist)

# Hybrid similarity measure: average of cosine and normalized Euclidean
hybrid_sim = (cosine_sim + normalized_euclidean_sim) / 2

# Now you can use hybrid_sim for recommendations similar to previous examples


In [168]:
# Outlier courses (wrong descriptions)

courses = courses.drop(91)
courses = courses.reset_index(drop=True)

In [169]:
courses['course_id'] = courses.index

In [195]:
def recommend_courses(course_id, similarity_matrix, top_n=5):
    course_index = courses[courses['course_id'] == course_id].index[0]
    similarity_scores = list(enumerate(similarity_matrix[course_index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    recommended_course_indices = [i[0] for i in similarity_scores[1:top_n+1]]
    return courses.iloc[recommended_course_indices]

# Example usage: Recommend courses similar to course_id 1 based on hybrid similarity
course_number = random.randint(0, len(courses))
print(courses['name'][course_number])
similar_courses = recommend_courses(course_number, hybrid_sim, )
print(similar_courses[['course_id', 'name', 'merged_description']])

Introduction to Systematic Review and Meta-Analysis
     course_id                                              name  \
106        106                          AI for Medical Diagnosis   
209        209                                     Dentistry 101   
160        160  Introduction to Chemistry:  Reactions and Ratios   
251        251                        Getting Started with Azure   
198        198                       Data-driven Decision Making   

                                    merged_description  
106   \n AI is transforming the practice of medicin...  
209   \n Explore the role of dentists as healthcare...  
160   \n The focus and themes of the Introduction t...  
251   \n Navigating the Azure Portal and Creating C...  
198  One of the most important skills of successful...  


In [206]:
import random

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3))  # Considering unigrams, bigrams, and trigrams
tfidf_matrix = tfidf_vectorizer.fit_transform(courses['processed_description'])

# Cosine Similarity
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Example: Find courses similar to course_id 1
def get_similar_courses(course_id, similarity_matrix, courses_df):
    # Get course index
    course_idx = courses_df.index[courses_df['course_id'] == course_id].tolist()[0]
    # Get similarity scores and pair with course index
    similarity_scores = list(enumerate(similarity_matrix[course_idx]))
    # Sort the courses based on similarity scores
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the 5 most similar courses (excluding itself)
    most_similar_courses = similarity_scores[1:6]
    # Get course ids
    similar_course_ids = [courses_df.iloc[i[0]]['course_id'] for i in most_similar_courses]
    return similar_course_ids

# Get similar courses to a given course_id

course_number = random.randint(0, len(courses))
print(courses['name'][course_number])
similar_courses = get_similar_courses(course_number, cosine_sim_matrix, courses)
print(courses.loc[courses['course_id'].isin(similar_courses), ['name', 'course_id']])

Data Science Math Skills
                                                  name  course_id
46                              Tools for Data Science         46
55              Introduction to Data Science in Python         55
58                                SQL for Data Science         58
130  Biology Meets Programming: Bioinformatics for ...        130
213                      Applied Data Science Capstone        213
